In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# FER-2013 데이터셋을 불러오는 함수
def load_fer2013():
    data = pd.read_csv('C:/Users/user/Downloads/emotion_recognition/fer2013.csv')
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []

    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        faces.append(face.astype('float32'))

    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).values
    return faces, emotions

# FER-2013 데이터셋 로드
faces, emotions = load_fer2013()

# 데이터 전처리
faces = faces / 255.0

# 데이터를 학습 및 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.2, random_state=42)

# 간단한 CNN 모델 정의
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 훈련
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

# 모델 저장
model.save('fer2013_model.h5')

Epoch 1/10
404/404 [==============================] - 16s 38ms/step - loss: 1.7036 - accuracy: 0.3199 - val_loss: 1.5665 - val_accuracy: 0.4002
Epoch 2/10
404/404 [==============================] - 15s 38ms/step - loss: 1.4663 - accuracy: 0.4377 - val_loss: 1.4135 - val_accuracy: 0.4584
Epoch 3/10
404/404 [==============================] - 15s 38ms/step - loss: 1.3395 - accuracy: 0.4921 - val_loss: 1.3393 - val_accuracy: 0.4918
Epoch 4/10
404/404 [==============================] - 16s 39ms/step - loss: 1.2568 - accuracy: 0.5206 - val_loss: 1.2938 - val_accuracy: 0.5138
Epoch 5/10
404/404 [==============================] - 16s 39ms/step - loss: 1.1873 - accuracy: 0.5509 - val_loss: 1.2867 - val_accuracy: 0.5145
Epoch 6/10
404/404 [==============================] - 16s 39ms/step - loss: 1.1333 - accuracy: 0.5740 - val_loss: 1.2815 - val_accuracy: 0.5225
Epoch 7/10
404/404 [==============================] - 16s 40ms/step - loss: 1.0789 - accuracy: 0.5964 - val_loss: 1.2347 - val_accuracy:

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
import cv2
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# fer2013 데이터셋 로드
# 데이터셋은 https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data 에서 다운로드 가능
# 다운로드한 fer2013.csv 파일을 사용하세요.
# (주의: 파일 경로 및 구성은 실제 데이터셋에 따라 조절해야 할 수 있습니다.)
data = pd.read_csv('fer2013.csv')

# 픽셀 값 추출 및 이미지 크기 조정
pixels = data['pixels'].tolist()
images = np.array([np.fromstring(pixel, dtype=int, sep=' ').reshape((48, 48, 1)) for pixel in pixels])
images = images / 255.0  # 정규화

# 레이블 추출
labels = data['emotion'].values

# 레이블 인코딩
le = LabelEncoder()
labels = le.fit_transform(labels)

# 레이블을 one-hot 인코딩으로 변환
labels = to_categorical(labels)

# 훈련 및 검증 데이터로 나누기
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# 모델 정의
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(7, activation='softmax')  # 클래스 수는 7개 (감정 종류)
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 데이터 증강을 사용한 생성기 정의
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

# 훈련 데이터 생성기 생성
train_gen = datagen.flow(X_train, y_train, batch_size=64)

# 모델 훈련
model.fit(train_gen, epochs=30, validation_data=(X_val, y_val))

# 훈련된 모델 저장
model.save_weights('fer2013_model_weights.h5')

Epoch 1/30
449/449 [==============================] - 20s 42ms/step - loss: 1.7971 - accuracy: 0.2548 - val_loss: 1.7321 - val_accuracy: 0.2907
Epoch 2/30
449/449 [==============================] - 19s 42ms/step - loss: 1.7402 - accuracy: 0.2824 - val_loss: 1.7201 - val_accuracy: 0.2825
Epoch 3/30
449/449 [==============================] - 19s 42ms/step - loss: 1.6629 - accuracy: 0.3295 - val_loss: 1.5750 - val_accuracy: 0.3940
Epoch 4/30
449/449 [==============================] - 20s 44ms/step - loss: 1.5675 - accuracy: 0.3850 - val_loss: 1.4613 - val_accuracy: 0.4486
Epoch 5/30
449/449 [==============================] - 20s 45ms/step - loss: 1.4903 - accuracy: 0.4222 - val_loss: 1.3686 - val_accuracy: 0.4777
Epoch 6/30
449/449 [==============================] - 20s 44ms/step - loss: 1.4316 - accuracy: 0.4465 - val_loss: 1.3951 - val_accuracy: 0.4742
Epoch 7/30
449/449 [==============================] - 20s 44ms/step - loss: 1.4006 - accuracy: 0.4617 - val_loss: 1.3000 - val_accuracy: